In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Loading the data

In [ ]:
df_train= pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
df_test= pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [ ]:
df_train.head()

In [ ]:
df_train.drop("location", inplace=True, axis=1)
df_test.drop("location", inplace=True, axis=1)

Lets start with lowercasing, droping numebrs, removing punctuation and white spaces from both datasets

In [ ]:
import re
import string
for i in range(df_train["text"].size):
    df_train["text"][i]=df_train["text"][i].lower()
    df_train["text"][i]=re.sub(r'\d+', "", df_train["text"][i])

In [ ]:
for i in range(df_train["text"].size):
    result=df_train["text"][i].translate(str.maketrans('','', string.punctuation))
    df_train["text"][i]=result
    df_train["text"][i]=df_train["text"][i].strip()

In [ ]:
import re
import string
for i in range(df_test["text"].size):
    df_test["text"][i]=df_test["text"][i].lower()
    df_test["text"][i]=re.sub(r'\d+', "", df_test["text"][i])

In [ ]:
for i in range(df_test["text"].size):
    result=df_test["text"][i].translate(str.maketrans('','', string.punctuation))
    df_test["text"][i]=result
    df_test["text"][i]=df_test["text"][i].strip()

In [ ]:
df_test.head(7)

Now lets begin with tokenizing and removing stopwords.

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

stemmer = PorterStemmer()
for i in range(df_train["text"].size):
    traintokenz=word_tokenize(df_train["text"][i])     #tokenizing each tweet
    for word in traintokenz:
        if word in stopwords.words('english'):       #removing stopwords from tokens
            traintokenz.remove(word)
    for i in range(len(traintokenz)):
            traintokenz[i] = lemmatizer.lemmatize(traintokenz[i])
            traintokenz[i] = stemmer.stem(traintokenz[i])    # stemming the tokens
    df_train["text"][i]=" ".join(traintokenz)      #back to the dataframe, the converted string

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

for i in range(df_test["text"].size):
    traintokenz=word_tokenize(df_test["text"][i])    #tokenizing each tweet
    for word in traintokenz:
        if word in stopwords.words('english'):       # removing stopwords from tokens
            traintokenz.remove(word)
    for i in range(len(traintokenz)):
            traintokenz[i] = lemmatizer.lemmatize(traintokenz[i])  #lemmatizing
            traintokenz[i] = stemmer.stem(traintokenz[i])     #stemming tokens
            
    df_test["text"][i]=" ".join(traintokenz)

Let's fill the nan values in keywords and try to one-hot encode the column

In [ ]:
values={"id","keyword"}
train_keywords=df_train[values]
test_keywords=df_test[values]

In [ ]:
train_keywords["keyword"].fillna(value=train_keywords["keyword"].mode(1),inplace=True)
test_keywords["keyword"].fillna(value=train_keywords["keyword"].mode(1),inplace=True)

In [ ]:
test_keywords

In [ ]:
train_keywords=pd.get_dummies(train_keywords,columns = ["keyword"])
test_keywords=pd.get_dummies(test_keywords,columns=["keyword"])

In [ ]:
df_train_1=df_train.drop("keyword",axis=1,inplace=False)
df_test_1=df_test.drop("keyword",axis=1,inplace=False)

In [ ]:
x_train=df_train_1.drop("target",axis=1,inplace=False)
y_train=df_train_1["target"]

**CountVectorizer**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
matrix = CountVectorizer(max_features=12000)
Xtrain = matrix.fit_transform(x_train["text"]).toarray()
Xtest = matrix.transform(df_test_1["text"]).toarray()

In [ ]:
df2train=pd.concat([train_keywords, pd.DataFrame(Xtrain)], axis=1)
df2test=pd.concat([test_keywords, pd.DataFrame(Xtest)], axis=1)

In [ ]:
X_train_new,X_test_new=TFIDF(x_train["text"],df_test_1["text"])


Train-tes split

In [ ]:
from sklearn import metrics
from sklearn.model_selection import train_test_split

x_new_train, x_new_test, y_new_train, y_new_test = train_test_split(df2train, y_train, test_size=0.2)

In [ ]:
from sklearn import metrics
from sklearn.model_selection import train_test_split

x_new_train, x_new_test, y_new_train, y_new_test = train_test_split(Xtrain, y_train, test_size=0.2)

Deploying a simple ReLU, Softmax neural net

In [ ]:
x_new_train.shape

In [ ]:
from keras.layers import  Dropout, Dense
from keras.models import Sequential
from sklearn import metrics

In [ ]:
def Build_Model_DNN_Text(shape, nClasses, dropout=0.5):
    """
    buildModel_DNN_Tex(shape, nClasses,dropout)
    Build Deep neural networks Model for text classification
    Shape is input feature space
    nClasses is number of classes
    """
    model = Sequential()
    node = 12000 # number of nodes
    nLayers = 4 # number of  hidden layer
    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(nClasses, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [ ]:
model_DNN = Build_Model_DNN_Text(x_new_train.shape[1], 20)


In [ ]:
model_DNN.fit(x_new_train,y_new_train, validation_data=(x_new_test,y_new_test), epochs=10,batch_size=200)

In [ ]:
# Naive Bayes 
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(x_new_train, y_new_train)

In [ ]:
y_pred = classifier.predict(x_new_test)
print(metrics.classification_report(y_new_test, y_pred))


In [ ]:
print(x_new_train.shape)

GradientBoosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model=GradientBoostingClassifier(n_estimators=20)
model.fit(x_new_train, y_new_train)

In [ ]:
y_pred = model.predict(x_new_test)
print(metrics.classification_report(y_new_test, y_pred))


In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
model2=BaggingClassifier()
model2.fit(x_new_train,y_new_train)

In [ ]:
y_pred = model2.predict(x_new_test)
print(metrics.classification_report(y_new_test, y_pred))


In [ ]:
from sklearn.svm import LinearSVC
model3= LinearSVC(C=0.1, loss='hinge',max_iter=20000)
model3.fit(x_new_train,y_new_train)

In [ ]:
y_pred = model3.predict(x_new_test)
print(metrics.classification_report(y_new_test, y_pred))


In [ ]:
from sklearn import tree
model4=tree.DecisionTreeClassifier()
model4.fit(x_new_train,y_new_train)

In [ ]:
y_pred=model4.predict(x_new_test)
print(metrics.classification_report(y_new_test, y_pred))


In [ ]:
y_pred = classifier.predict(x_new_test)
print(metrics.classification_report(y_new_test, y_pred))


In [ ]:
y_2_pred=model.predict(x_new_test)
print(metrics.classification_report(y_new_test, y_2_pred))


***Conclusion***


After evaluating each model on the train-test split dataset, and fine tuning some regularization terms, we can argue that the best model among these is the "Support Vector Classifier" and now we predict the test data and submit it.

In [ ]:
from sklearn.svm import LinearSVC
model3= LinearSVC(C=0.2,loss='hinge')
model3.fit(Xtrain,y_train)

In [ ]:
y_pred=model3.predict(Xtest)

In [ ]:
d={'id':df_test_1["id"] ,'target':y_pred}
Mub=pd.DataFrame(d)
Mub.to_csv('/kaggle/working/Mub.csv',index=False)


